# Advanced Querying Mongo

Importing libraries and setting up connection

In [232]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies",connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db=client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [1]:
# Your Code
query={"name":"Babelgum"}
list(db.companies.find(query))

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [35]:
# Your Code
from pymongo import ASCENDING, DESCENDING
query={'number_of_employees':{"$gt": 5000 }}
list(db.companies.find(query,{'number_of_employees':1, "name":1}).sort(("number_of_employees"),DESCENDING).limit(20))

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [2]:
# Your Code
query={"$and":[{'founded_year':{"$gte": 2000 }},{'founded_year':{"$lte": 2005 }}]}
list(db.companies.find(query,{'founded_year':1, "name":1}))

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [85]:
# Your Code
query={"$and":[{"ipo.valuation_amount":{"$gte":100000000}},{'founded_year':{"$lte": 2010 }}]}
list(db.companies.find(query,{'founded_year':1,"ipo.valuation_amount":1}).limit(5))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'founded_year': 2004,
  'ipo': {'valuation_amount': 104000000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'founded_year': 2006,
  'ipo': {'valuation_amount': 18100000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'founded_year': 2004,
  'ipo': {'valuation_amount': 1300000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e0c'),
  'founded_year': 2003,
  'ipo': {'valuation_amount': 9310000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e81'),
  'founded_year': 2004,
  'ipo': {'valuation_amount': 290000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [5]:
# Your Code
query={"$and":[{'founded_year':{"$lte": 2005 }},{'number_of_employees':{"$lte": 1000 }}]}
list(db.companies.find(query,{'founded_year':1, "name":1}).sort(("number_of_employees"),DESCENDING))

### 6. All the companies that don't include the `partners` field.

In [78]:
# Your Code
list(db.companies.find({"partners":{"$exists":False}}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [6]:
# Your Code
list(db.companies.find({"category_code":{"$type":10}}).limit(7))

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [7]:
# Your Code
query={"$and":[{'number_of_employees':{"$gte": 100 }},{'number_of_employees':{"$lt": 1000 }}]}
list(db.companies.find(query,{'name':1, "number_of_employees":1}))

### 9. Order all the companies by their IPO price in a descending order.

In [8]:
# Your Code
list(db.companies.find().sort("ipo.valuation_amount",DESCENDING).limit(5))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [9]:
# Your Code
list(db.companies.find().sort("number_of_employees",DESCENDING).limit(10))

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [10]:
# Your Code
query={"$and":[{'founded_month':{"$gte": 7 }},{'founded_month':{"$lt": 12 }}]}
list(db.companies.find(query).limit(2))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [11]:
# Your Code
query={"$and":[{'founded_year':{"$lt": 2000 }},{'acquisition.price_amount':{"$gt": 1000000 }}]}
list(db.companies.find(query).limit(2))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [12]:
# Your Code
query={"acquisition.acquired_year":{"$gte": 2010 }}
list(db.companies.find(query,{"name":1, "acquisition":1}).sort("acquisition.price_amount").limit(2))

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [132]:
# Your Code
list(db.companies.find({},{"name":1, "founded_year":1}).sort("founded_year", DESCENDING).limit(2))

[{'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [13]:
# Your Code
query={"$and":[{'founded_day':{"$gte": 1 }},{'founded_day':{"$lte": 7 }}]}
list(db.companies.find(query).sort("acquisition.price_amount",DESCENDING).limit(10))

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [14]:
# Your Code
query={"$and":[{'category_code':"web"},{'number_of_employees':{"$gte": 4000}}]}
list(db.companies.find(query).sort("number_of_employees",ASCENDING).limit(10))

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [1]:
# Your Code
query={"$and":[{'acquisition.price_currency_code':"EUR"},{'acquisition.price_amount':{"$gte": 10000000}}]}
list(db.companies.find(query).limit(10))

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [2]:
# Your Code
query={"$and":[{'acquisition.acquired_month':{"$gte": 1 }},{'acquisition.acquired_month':{"$lte": 3 }}]}
list(db.companies.find(query,{"name":1,"acquisition":1}).limit(2))

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [3]:
# Your Code
query={"$and":[{'founded_year':{"$gte": 2000 }},{'founded_year':{"$lte": 2010 }},{"acquisition.acquired_year":{"$not":{"$lt":2011}}}]}
list(db.companies.find(query).limit(1))

### 20. All the companies that have been 'deadpooled' after the third year.

In [284]:
#where permite leer codigo javascript, y se usa el this para hacer referencia a los documentos dentro del string,

In [282]:
query={"$and":[{"$where" : "this.deadpooled_year > (this.founded_year)+3" },{"founded_year":{"$ne":None}}]
        } 

In [ ]:
list(db.companies.find(query,{"deadpooled_year":1,"founded_year":1}))